# Sales Forecasting with Rasgo

This notebook shows how to perform the data preparation and feature engineering for a sales forecasting model. Starting with [AdventureWorks](https://docs.microsoft.com/en-us/sql/samples/adventureworks-install-configure) data preloaded in Rasgo, the data will be explored, features created and modeling data extracted.

This analysis will be focused on the internet sales for this company.

## Packages

The documentation for each packaged used in this tutorial is linked below:
* [numpy](https://numpy.org/doc/stable/)
* [os](https://docs.python.org/3/library/os.html)
* [pandas](https://pandas.pydata.org/docs/)
* [pyrasgo](https://docs.rasgoml.com/rasgo-docs/)
* [scikit-learn](https://scikit-learn.org/stable/)
    * [sklearn.metrics](https://scikit-learn.org/stable/modules/model_evaluation.html)
* [XGBoost](https://xgboost.readthedocs.io/en/latest/)

### Ensure that pyrasgo is installed and up to date

In [1]:
!pip install pyrasgo --upgrade

!pip show pyrasgo

Name: pyrasgo
Version: 0.4.9
Summary: Alpha version of the Rasgo Python interface.
Home-page: https://www.rasgoml.com/
Author: Patrick Dougherty
Author-email: patrick@rasgoml.com
License: GNU Affero General Public License v3 or later (AGPLv3+)
Location: /Users/nick/Git/workspace/venv/lib/python3.9/site-packages
Requires: idna, more-itertools, pandas, pyarrow, pydantic, pyyaml, requests, snowflake-connector-python, snowflake-connector-python, tqdm
Required-by: 


In [2]:
import numpy as np
import os
import pandas as pd
import 
from sklearn.metrics import mean_squared_error
import           as xgb

## Create account on Rasgo

## Access Rasgo

### Create account

Next, click [here](https://app.rasgoml.com/account/register) to create an account on the Rasgo UI. Fill in the required information on the web page.

<p align="center">
  <img src="img/RasgoAccountRegistration.png" alt="Rasgo Account Registration" width="512">
</p>

You can close the browser tab as you will receive an email from rasgo to verify your email address. Click the **Verify Email** button to verify.

<p align="center">
  <img src="img/RasgoWelcome.png" alt="Verify Email" width="390">
</p>

This will open browser tab where you can log into the UI.

### Log into Rasgo UI

Enter your username and password and click **Login**.

<p align="center">
  <img src="img/RasgoLogin.png" alt="Login to Rasgo" width="528">
</p>

to be taken to the Rasgo App homepage.

### Copy your API Key

Click the **API KEY** button in the upper right of the screen

<img src="img/APIKEY.png" alt="Copy API Key" width="128">

to copy your API key to the clipboard.

### Save API Key as an environment variable

Save the API Key as an environment variable called **RASGO_API_KEY**. This can be done on:
* [Linux](https://unix.stackexchange.com/questions/21598/how-do-i-set-a-user-environment-variable-permanently-not-session)
* [Mac](https://apple.stackexchange.com/questions/395457/how-to-set-environment-variable-permanently-on-macos-catalina)
* [Windows](https://stackoverflow.com/questions/17312348/how-do-i-set-windows-environment-variables-permanently)

## Work with PyRasgo

### Load the API Key from the environment variable and connect to the Data Warehouse

In [18]:
API_KEY = os.getenv('RASGO_API_KEY')

rasgo = pyrasgo.         (API_KEY)

### Get list of available datasets

Loop over all available datasets and print out the dataset ID and Name.

In [4]:
datasets = sorted(rasgo.get.          (), key=lambda x: x._api_dataset.id)
for ds in datasets:
    print(f"ID: {ds.id}\tDataset: {ds.name}")

ID: 52	Dataset: Adventureworks: Dim Account
ID: 53	Dataset: Adventureworks: Dim Currency
ID: 54	Dataset: Adventureworks: Dim Salesreason
ID: 55	Dataset: Adventureworks: Dim Customer
ID: 56	Dataset: Adventureworks: Dim Promotion
ID: 57	Dataset: Adventureworks: Dim Date
ID: 58	Dataset: Adventureworks: Dim Productcategory
ID: 59	Dataset: Adventureworks: Dim Departmentgroup
ID: 60	Dataset: Adventureworks: Newfact Currencyrate
ID: 61	Dataset: Adventureworks: Dim Geography
ID: 62	Dataset: Adventureworks: Dim Organization
ID: 63	Dataset: Adventureworks: Dim Reseller
ID: 64	Dataset: Adventureworks: Fact Additionalinternationalproductdescription
ID: 65	Dataset: Adventureworks: Fact Productinventory
ID: 66	Dataset: Adventureworks: Fact Callcenter
ID: 67	Dataset: Adventureworks: Fact Resellersales
ID: 68	Dataset: Adventureworks: Fact Internetsalesreason
ID: 69	Dataset: Adventureworks: Fact Salesquota
ID: 70	Dataset: Adventureworks: Prospectivebuyer
ID: 71	Dataset: Adventureworks: Dim Scenario
ID:

The two datasets of interest are 74 containing the sales information and 56 containing details about any promotions in place.

### Examine Internet Sales

In [5]:
internet_sales = rasgo.get.dataset(   )
internet_sales.preview()

,PRODUCTKEY,ORDERDATEKEY,DUEDATEKEY,SHIPDATEKEY,CUSTOMERKEY,PROMOTIONKEY,CURRENCYKEY,SALESTERRITORYKEY,SALESORDERNUMBER,SALESORDERLINENUMBER,...,PRODUCTSTANDARDCOST,TOTALPRODUCTCOST,SALESAMOUNT,TAXAMT,FREIGHT,CARRIERTRACKINGNUMBER,CUSTOMERPONUMBER,ORDERDATE,DUEDATE,SHIPDATE
0,310,20101229,20110110,20110105,21768,1,19,6,SO43697,1,...,2171.2942,2171.2942,3578.2700,286.2616,89.4568,None,None,2010-12-29,2011-01-10,2011-01-05
1,346,20101229,20110110,20110105,28389,1,39,7,SO43698,1,...,1912.1544,1912.1544,3399.9900,271.9992,84.9998,None,None,2010-12-29,2011-01-10,2011-01-05
2,346,20101229,20110110,20110105,25863,1,100,1,SO43699,1,...,1912.1544,1912.1544,3399.9900,271.9992,84.9998,None,None,2010-12-29,2011-01-10,2011-01-05
3,336,20101229,20110110,20110105,14501,1,100,4,SO43700,1,...,413.1463,413.1463,699.0982,55.9279,17.4775,None,None,2010-12-29,2011-01-10,2011-01-05
4,346,20101229,20110110,20110105,11003,1,6,9,SO43701,1,...,1912.1544,1912.1544,3399.9900,271.9992,84.9998,None,None,2010-12-29,2011-01-10,2011-01-05
5,311,20101230,20110111,20110106,27645,1,100,4,SO43702,1,...,2171.2942,2171.2942,3578.2700,286.2616,89.4568,None,None,2010-12-30,2011-01-11,2011-01-06
6,310,20101230,20110111,20110106,16624,1,6,9,SO43703,1,...,2171.2942,2171.2942,3578.2700,286.2616,89.4568,None,None,2010-12-30,2011-01-11,2011-01-06
7,351,20101230,20110111,20110106,11005,1,6,9,SO43704,1,...,1898.0944,1898.0944,3374.9900,269.9992,84.3748,None,None,2010-12-30,2011-01-11,2011-01-06
8,344,20101230,20110111,20110106,11011,1,6,9,SO43705,1,...,1912.1544,1912.1544,3399.9900,271.9992,84.9998,None,None,2010-12-30,2011-01-11,2011-01-06
9,312,20101231,20110112,20110107,27621,1,100,4,SO43706,1,...,2171.2942,2171.2942,3578.2700,286.2616,89.4568,None,None,2010-12-31,2011-01-12,2011-01-07


This looks promising, but I'd like to see a single product sorted by date. This can be done through the use of the filter and order transforms. To use filter, the product we want to filter on is needed, as we don't know that yet, we will just order by *PRODUCTKEY* and *ORDERDATE*.

In [6]:
internet_sales.order(col_list=['PRODUCTKEY',              ], order_method="ASC").preview()

,PRODUCTKEY,ORDERDATEKEY,DUEDATEKEY,SHIPDATEKEY,CUSTOMERKEY,PROMOTIONKEY,CURRENCYKEY,SALESTERRITORYKEY,SALESORDERNUMBER,SALESORDERLINENUMBER,...,PRODUCTSTANDARDCOST,TOTALPRODUCTCOST,SALESAMOUNT,TAXAMT,FREIGHT,CARRIERTRACKINGNUMBER,CUSTOMERPONUMBER,ORDERDATE,DUEDATE,SHIPDATE
0,214,20121228,20130109,20130104,12132,1,100,7,SO51181,4,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-28,2013-01-09,2013-01-04
1,214,20121228,20130109,20130104,16313,1,100,8,SO51180,4,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-28,2013-01-09,2013-01-04
2,214,20121229,20130110,20130105,12390,1,100,8,SO51191,4,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-29,2013-01-10,2013-01-05
3,214,20121229,20130110,20130105,11241,1,100,7,SO51192,2,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-29,2013-01-10,2013-01-05
4,214,20121230,20130111,20130106,11338,1,100,8,SO51207,4,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-30,2013-01-11,2013-01-06
5,214,20121230,20130111,20130106,24604,1,6,9,SO51212,4,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-30,2013-01-11,2013-01-06
6,214,20121231,20130112,20130107,11061,1,6,9,SO51237,4,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-31,2013-01-12,2013-01-07
7,214,20121231,20130112,20130107,25625,1,100,8,SO51246,4,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-31,2013-01-12,2013-01-07
8,214,20121231,20130112,20130107,11615,1,98,10,SO51232,2,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-31,2013-01-12,2013-01-07
9,214,20121231,20130112,20130107,28204,1,6,9,SO51234,2,...,13.0863,13.0863,34.99,2.7992,0.8748,None,None,2012-12-31,2013-01-12,2013-01-07


This looks reasonable, use this for our modeling. For future reference, what columns exist in this table?

In [7]:
internet_sales.preview().         .sort_values()

Index(['CARRIERTRACKINGNUMBER', 'CURRENCYKEY', 'CUSTOMERKEY',
       'CUSTOMERPONUMBER', 'DISCOUNTAMOUNT', 'DUEDATE', 'DUEDATEKEY',
       'EXTENDEDAMOUNT', 'FREIGHT', 'ORDERDATE', 'ORDERDATEKEY',
       'ORDERQUANTITY', 'PRODUCTKEY', 'PRODUCTSTANDARDCOST', 'PROMOTIONKEY',
       'REVISIONNUMBER', 'SALESAMOUNT', 'SALESORDERLINENUMBER',
       'SALESORDERNUMBER', 'SALESTERRITORYKEY', 'SHIPDATE', 'SHIPDATEKEY',
       'TAXAMT', 'TOTALPRODUCTCOST', 'UNITPRICE', 'UNITPRICEDISCOUNTPCT'],
      dtype='object')

*PROMOTIONKEY* is probably important for a sales forecast. Promotion information can be found in dataset 56.

In [8]:
promotion = rasgo.get.dataset(    )
promotion.preview()

,PROMOTIONKEY,PROMOTIONALTERNATEKEY,ENGLISHPROMOTIONNAME,SPANISHPROMOTIONNAME,FRENCHPROMOTIONNAME,DISCOUNTPCT,ENGLISHPROMOTIONTYPE,SPANISHPROMOTIONTYPE,FRENCHPROMOTIONTYPE,ENGLISHPROMOTIONCATEGORY,SPANISHPROMOTIONCATEGORY,FRENCHPROMOTIONCATEGORY,STARTDATE,ENDDATE,MINQTY,MAXQTY
0,1,1,No Discount,Sin descuento,Aucune remise,0.00,No Discount,Sin descuento,Aucune remise,No Discount,Sin descuento,Aucune remise,2010-11-29,2014-06-30,0,NaN
1,2,2,Volume Discount 11 to 14,Descuento por volumen (entre 11 y 14),Remise sur quantité (de 11 à 14),0.02,Volume Discount,Descuento por volumen,Remise sur quantité,Reseller,Distribuidor,Revendeur,2010-12-29,2013-12-28,11,14.0
2,3,3,Volume Discount 15 to 24,Descuento por volumen (entre 15 y 24),Remise sur quantité (de 15 à 24),0.05,Volume Discount,Descuento por volumen,Remise sur quantité,Reseller,Distribuidor,Revendeur,2010-12-29,2013-12-28,15,24.0
3,4,4,Volume Discount 25 to 40,Descuento por volumen (entre 25 y 40),Remise sur quantité (de 25 à 40),0.10,Volume Discount,Descuento por volumen,Remise sur quantité,Reseller,Distribuidor,Revendeur,2010-12-29,2013-12-28,25,40.0
4,5,5,Volume Discount 41 to 60,Descuento por volumen (entre 41 y 60),Remise sur quantité (de 41 à 60),0.15,Volume Discount,Descuento por volumen,Remise sur quantité,Reseller,Distribuidor,Revendeur,2010-12-29,2013-12-28,41,60.0
5,6,6,Volume Discount over 60,Descuento por volumen (más de 60),Remise sur quantité (au-delà de 60),0.20,Volume Discount,Descuento por volumen,Remise sur quantité,Reseller,Distribuidor,Revendeur,2010-12-29,2013-12-28,61,NaN
6,7,7,Mountain-100 Clearance Sale,"Liquidación de bicicleta de montaña, 100",Liquidation VTT 100,0.35,Discontinued Product,Descatalogado,Ce produit n'est plus commercialisé,Reseller,Distribuidor,Revendeur,2011-11-12,2011-12-28,0,NaN
7,8,8,Sport Helmet Discount-2002,"Casco deportivo, descuento: 2002",Remise sur les casques sport - 2002,0.10,Seasonal Discount,Descuento de temporada,Remise saisonnière,Reseller,Distribuidor,Revendeur,2011-12-29,2012-01-28,0,NaN
8,9,9,Road-650 Overstock,"Bicicleta de carretera: 650, oferta especial",Déstockage Vélo de route 650,0.30,Excess Inventory,Inventario excedente,Déstockage,Reseller,Distribuidor,Revendeur,2011-12-29,2012-02-28,0,NaN
9,10,10,Mountain Tire Sale,Oferta de cubierta de montaña,Vente de pneus de VTT,0.50,Excess Inventory,Inventario excedente,Déstockage,Customer,Cliente,Client,2012-12-12,2013-02-26,0,NaN


## Sales Data

Work with the sales and promotion data to create the base modeling time-series features for the sales forecasting model.

### Merge Promo data

First, we want to clean up the promotion data to only keep what needs to be added to the sales data. Drop all columns except *PROMOTIONKEY* and *DISCOUNTPCT* from  promotion using the `drop_columns` transformation.

In [9]:
reduced_promo = promotion.                (include_cols=[                , 'DISCOUNTPCT'])
reduced_promo.order(col_list=['PROMOTIONKEY'], order_method="ASC").preview()

,PROMOTIONKEY,DISCOUNTPCT
0,1,0.00
1,2,0.02
2,3,0.05
3,4,0.10
4,5,0.15
5,6,0.20
6,7,0.35
7,8,0.10
8,9,0.30
9,10,0.50


Now merge this with the internet sales datausing the `join` transformation.

In [10]:
sales_promo = reduced_promo.join(join_table=internet_sales,
                                 join_type=       ,
                                 join_columns={'PROMOTIONKEY':'PROMOTIONKEY'})
sales_promo.order(col_list=['PRODUCTKEY', 'ORDERDATE'], order_method=      ).preview()

,PROMOTIONKEY,DISCOUNTPCT,CUSTOMERKEY,DUEDATE,ORDERQUANTITY,CURRENCYKEY,UNITPRICE,SHIPDATEKEY,CARRIERTRACKINGNUMBER,ORDERDATE,...,SALESORDERLINENUMBER,REVISIONNUMBER,SHIPDATE,TOTALPRODUCTCOST,CUSTOMERPONUMBER,UNITPRICEDISCOUNTPCT,ORDERDATEKEY,SALESAMOUNT,TAXAMT,PRODUCTSTANDARDCOST
0,1,0.0,12132,2013-01-09,1,100,34.99,20130104,None,2012-12-28,...,4,1,2013-01-04,13.0863,None,0.0,20121228,34.99,2.7992,13.0863
1,1,0.0,16313,2013-01-09,1,100,34.99,20130104,None,2012-12-28,...,4,1,2013-01-04,13.0863,None,0.0,20121228,34.99,2.7992,13.0863
2,1,0.0,11241,2013-01-10,1,100,34.99,20130105,None,2012-12-29,...,2,1,2013-01-05,13.0863,None,0.0,20121229,34.99,2.7992,13.0863
3,1,0.0,12390,2013-01-10,1,100,34.99,20130105,None,2012-12-29,...,4,1,2013-01-05,13.0863,None,0.0,20121229,34.99,2.7992,13.0863
4,1,0.0,24604,2013-01-11,1,6,34.99,20130106,None,2012-12-30,...,4,1,2013-01-06,13.0863,None,0.0,20121230,34.99,2.7992,13.0863
5,1,0.0,11338,2013-01-11,1,100,34.99,20130106,None,2012-12-30,...,4,1,2013-01-06,13.0863,None,0.0,20121230,34.99,2.7992,13.0863
6,1,0.0,11061,2013-01-12,1,6,34.99,20130107,None,2012-12-31,...,4,1,2013-01-07,13.0863,None,0.0,20121231,34.99,2.7992,13.0863
7,1,0.0,28204,2013-01-12,1,6,34.99,20130107,None,2012-12-31,...,2,1,2013-01-07,13.0863,None,0.0,20121231,34.99,2.7992,13.0863
8,1,0.0,11615,2013-01-12,1,98,34.99,20130107,None,2012-12-31,...,2,1,2013-01-07,13.0863,None,0.0,20121231,34.99,2.7992,13.0863
9,1,0.0,25625,2013-01-12,1,100,34.99,20130107,None,2012-12-31,...,4,1,2013-01-07,13.0863,None,0.0,20121231,34.99,2.7992,13.0863


### Create Weekly Data

Now, we want to forecast these sales weekly, so we need to extract the week from the *ORDERDATE*. This can be done using the transform `datetrunc`.

In [11]:
salesds = sales_promo.datetrunc(dates={'ORDERDATE':         })
salesds.order(col_list=[              , 'ORDERDATE'], order_method="ASC").preview()

,DISCOUNTPCT,PROMOTIONKEY,CARRIERTRACKINGNUMBER,ORDERDATE,SALESTERRITORYKEY,EXTENDEDAMOUNT,PRODUCTKEY,SALESORDERNUMBER,CUSTOMERPONUMBER,UNITPRICEDISCOUNTPCT,...,ORDERQUANTITY,CURRENCYKEY,UNITPRICE,SHIPDATEKEY,ORDERDATEKEY,SHIPDATE,TOTALPRODUCTCOST,REVISIONNUMBER,PRODUCTSTANDARDCOST,ORDERDATE_WEEK
0,0.0,1,None,2013-08-18,1,8.99,479,SO64516,None,0.0,...,1,100,8.99,20130825,20130818,2013-08-25,3.3623,1,3.3623,2013-08-18
1,0.0,1,None,2013-08-18,8,8.99,479,SO64519,None,0.0,...,1,100,8.99,20130825,20130818,2013-08-25,3.3623,1,3.3623,2013-08-18
2,0.0,1,None,2013-08-18,4,8.99,479,SO64514,None,0.0,...,1,100,8.99,20130825,20130818,2013-08-25,3.3623,1,3.3623,2013-08-18
3,0.0,1,None,2013-08-18,1,8.99,479,SO64515,None,0.0,...,1,100,8.99,20130825,20130818,2013-08-25,3.3623,1,3.3623,2013-08-18
4,0.0,1,None,2013-08-19,4,8.99,479,SO64604,None,0.0,...,1,100,8.99,20130826,20130819,2013-08-26,3.3623,1,3.3623,2013-08-18
5,0.0,1,None,2013-08-19,6,8.99,479,SO64601,None,0.0,...,1,19,8.99,20130826,20130819,2013-08-26,3.3623,1,3.3623,2013-08-18
6,0.0,1,None,2013-08-19,9,8.99,479,SO64598,None,0.0,...,1,6,8.99,20130826,20130819,2013-08-26,3.3623,1,3.3623,2013-08-18
7,0.0,1,None,2013-08-19,10,8.99,479,SO64523,None,0.0,...,1,98,8.99,20130826,20130819,2013-08-26,3.3623,1,3.3623,2013-08-18
8,0.0,1,None,2013-08-19,1,8.99,479,SO64583,None,0.0,...,1,100,8.99,20130826,20130819,2013-08-26,3.3623,1,3.3623,2013-08-18
9,0.0,1,None,2013-08-19,4,8.99,479,SO64606,None,0.0,...,1,100,8.99,20130826,20130819,2013-08-26,3.3623,1,3.3623,2013-08-18


The new week column is called *ORDERDATE_WEEK*. This is clunky, so let's rename it to *ORDERWEEK* using the `rename` transformation.

In [12]:
newsalesds = salesds.        (renames={'ORDERDATE_WEEK': 'ORDERWEEK'})
newsalesds.order(col_list=['PRODUCTKEY', 'ORDERWEEK'], order_method="ASC").preview()

,ORDERWEEK,CARRIERTRACKINGNUMBER,CURRENCYKEY,DUEDATE,ORDERDATE,DISCOUNTAMOUNT,UNITPRICEDISCOUNTPCT,PRODUCTSTANDARDCOST,SHIPDATE,SHIPDATEKEY,...,SALESAMOUNT,DISCOUNTPCT,CUSTOMERKEY,PRODUCTKEY,SALESTERRITORYKEY,TAXAMT,REVISIONNUMBER,CUSTOMERPONUMBER,EXTENDEDAMOUNT,UNITPRICE
0,2012-12-23,None,100,2013-01-09,2012-12-28,0.0,0.0,13.0863,2013-01-04,20130104,...,34.99,0.0,12132,214,7,2.7992,1,None,34.99,34.99
1,2012-12-23,None,100,2013-01-09,2012-12-28,0.0,0.0,13.0863,2013-01-04,20130104,...,34.99,0.0,16313,214,8,2.7992,1,None,34.99,34.99
2,2012-12-23,None,100,2013-01-10,2012-12-29,0.0,0.0,13.0863,2013-01-05,20130105,...,34.99,0.0,11241,214,7,2.7992,1,None,34.99,34.99
3,2012-12-23,None,100,2013-01-10,2012-12-29,0.0,0.0,13.0863,2013-01-05,20130105,...,34.99,0.0,12390,214,8,2.7992,1,None,34.99,34.99
4,2012-12-30,None,6,2013-01-12,2012-12-31,0.0,0.0,13.0863,2013-01-07,20130107,...,34.99,0.0,28204,214,9,2.7992,1,None,34.99,34.99
5,2012-12-30,None,100,2013-01-12,2012-12-31,0.0,0.0,13.0863,2013-01-07,20130107,...,34.99,0.0,25625,214,8,2.7992,1,None,34.99,34.99
6,2012-12-30,None,98,2013-01-12,2012-12-31,0.0,0.0,13.0863,2013-01-07,20130107,...,34.99,0.0,11615,214,10,2.7992,1,None,34.99,34.99
7,2012-12-30,None,19,2013-01-13,2013-01-01,0.0,0.0,13.0863,2013-01-08,20130108,...,34.99,0.0,18544,214,6,2.7992,1,None,34.99,34.99
8,2012-12-30,None,100,2013-01-13,2013-01-01,0.0,0.0,13.0863,2013-01-08,20130108,...,34.99,0.0,11433,214,7,2.7992,1,None,34.99,34.99
9,2012-12-30,None,6,2013-01-14,2013-01-02,0.0,0.0,13.0863,2013-01-09,20130109,...,34.99,0.0,11456,214,9,2.7992,1,None,34.99,34.99


Alternatively, we can just chain these transformations together.

In [13]:
salesds = sales_promo.           (dates={'ORDERDATE': 'week'}).       (
                                renames={'ORDERDATE_WEEK': 'ORDERWEEK'})
salesds.order(col_list=['PRODUCTKEY', 'ORDERWEEK'], order_method="ASC").preview()

,ORDERWEEK,UNITPRICE,SALESTERRITORYKEY,TAXAMT,SALESAMOUNT,TOTALPRODUCTCOST,PRODUCTSTANDARDCOST,SHIPDATEKEY,CUSTOMERKEY,FREIGHT,...,ORDERDATE,CURRENCYKEY,ORDERQUANTITY,ORDERDATEKEY,UNITPRICEDISCOUNTPCT,PRODUCTKEY,PROMOTIONKEY,SALESORDERNUMBER,CUSTOMERPONUMBER,SHIPDATE
0,2014-01-26,9.99,4,0.7992,9.99,3.7363,3.7363,20140204,21163,0.2498,...,2014-01-28,100,1,20140128,0.0,478,1,SO75099,None,2014-02-04
1,2014-01-26,9.99,4,0.7992,9.99,3.7363,3.7363,20140202,13338,0.2498,...,2014-01-26,100,1,20140126,0.0,478,1,SO75059,None,2014-02-02
2,2014-01-26,9.99,8,0.7992,9.99,3.7363,3.7363,20140202,14933,0.2498,...,2014-01-26,100,1,20140126,0.0,478,1,SO75053,None,2014-02-02
3,2014-01-26,9.99,4,0.7992,9.99,3.7363,3.7363,20140202,22356,0.2498,...,2014-01-26,100,1,20140126,0.0,478,1,SO75042,None,2014-02-02
4,2014-01-26,9.99,1,0.7992,9.99,3.7363,3.7363,20140202,20473,0.2498,...,2014-01-26,100,1,20140126,0.0,478,1,SO75044,None,2014-02-02
5,2014-01-26,9.99,6,0.7992,9.99,3.7363,3.7363,20140204,13350,0.2498,...,2014-01-28,100,1,20140128,0.0,478,1,SO75100,None,2014-02-04
6,2014-01-26,9.99,4,0.7992,9.99,3.7363,3.7363,20140204,21717,0.2498,...,2014-01-28,100,1,20140128,0.0,478,1,SO75097,None,2014-02-04
7,2014-01-26,9.99,6,0.7992,9.99,3.7363,3.7363,20140203,16370,0.2498,...,2014-01-27,100,1,20140127,0.0,478,1,SO75077,None,2014-02-03
8,2014-01-26,9.99,1,0.7992,9.99,3.7363,3.7363,20140203,23273,0.2498,...,2014-01-27,100,1,20140127,0.0,478,1,SO75076,None,2014-02-03
9,2014-01-26,9.99,1,0.7992,9.99,3.7363,3.7363,20140203,22341,0.2498,...,2014-01-27,100,1,20140127,0.0,478,1,SO75078,None,2014-02-03


Now we can aggregate this to the product-week level and create aggregations of the *'DISCOUNTAMOUNT'*, *'DISCOUNTPCT'*, *'ORDERQUANTITY'*, *'PRODUCTSTANDARDCOST'*, *'SALESAMOUNT'*, *'TAXAMT'*, *'TOTALPRODUCTCOST'*, *'UNITPRICE'*, *'UNITPRICEDISCOUNTPCT'* using the `aggregate` transform.

In [14]:
salesds = sales_promo.datetrunc(dates={'ORDERDATE': 'week'}).rename(
                                renames={'ORDERDATE_WEEK': 'ORDERWEEK'}).aggregate(
                                group_by=['PRODUCTKEY', 'ORDERWEEK'],
                                aggregations={'DISCOUNTAMOUNT': [      , 'MAX', 'AVG', 'SUM'], 
                                              'DISCOUNTPCT': ['MIN',       , 'AVG', 'SUM'],
                                              'ORDERQUANTITY': ['SUM'],
                                              'PRODUCTSTANDARDCOST': [      , 'SUM'],
                                              'SALESAMOUNT': ['SUM'], 
                                              'TAXAMT': [     ],
                                              'TOTALPRODUCTCOST': ['AVG', 'SUM'],
                                              'UNITPRICE': ['AVG', 'SUM'],
                                              'UNITPRICEDISCOUNTPCT': ['MIN', 'MAX', 'AVG', 'SUM']})
salesds.order(col_list=['PRODUCTKEY', 'ORDERWEEK'], order_method="ASC").preview()

,PRODUCTKEY,ORDERWEEK,DISCOUNTAMOUNT_MIN,DISCOUNTAMOUNT_MAX,DISCOUNTAMOUNT_AVG,DISCOUNTAMOUNT_SUM,DISCOUNTPCT_MIN,DISCOUNTPCT_MAX,DISCOUNTPCT_AVG,DISCOUNTPCT_SUM,...,SALESAMOUNT_SUM,TAXAMT_SUM,TOTALPRODUCTCOST_AVG,TOTALPRODUCTCOST_SUM,UNITPRICE_AVG,UNITPRICE_SUM,UNITPRICEDISCOUNTPCT_MIN,UNITPRICEDISCOUNTPCT_MAX,UNITPRICEDISCOUNTPCT_AVG,UNITPRICEDISCOUNTPCT_SUM
0,214,2012-12-23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,139.9600,11.1968,13.0863000000,52.3452,34.9900000000,139.9600,0.0,0.0,0.0,0.0
1,214,2012-12-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,349.9000,27.9920,13.0863000000,130.8630,34.9900000000,349.9000,0.0,0.0,0.0,0.0
2,214,2013-01-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,419.8800,33.5904,13.0863000000,157.0356,34.9900000000,419.8800,0.0,0.0,0.0,0.0
3,214,2013-01-13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,209.9400,16.7952,13.0863000000,78.5178,34.9900000000,209.9400,0.0,0.0,0.0,0.0
4,214,2013-01-20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,384.8900,30.7912,13.0863000000,143.9493,34.9900000000,384.8900,0.0,0.0,0.0,0.0
5,214,2013-01-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1084.6900,86.7752,13.0863000000,405.6753,34.9900000000,1084.6900,0.0,0.0,0.0,0.0
6,214,2013-02-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1539.5600,123.1648,13.0863000000,575.7972,34.9900000000,1539.5600,0.0,0.0,0.0,0.0
7,214,2013-02-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1189.6600,95.1728,13.0863000000,444.9342,34.9900000000,1189.6600,0.0,0.0,0.0,0.0
8,214,2013-02-17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1154.6700,92.3736,13.0863000000,431.8479,34.9900000000,1154.6700,0.0,0.0,0.0,0.0
9,214,2013-02-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1294.6300,103.5704,13.0863000000,484.1931,34.9900000000,1294.6300,0.0,0.0,0.0,0.0


This gives us statistics for each product over a given week.

### Time-series feature engineering

For sales forcasting, in addition to the lagged variables, we need to know what the sales were in prior weeks. The transform `lag` can create these variables for us. In this case we will lag the following variables *'DISCOUNTAMOUNT_AVG'*, *'DISCOUNTPCT_AVG'*, *'ORDERQUANTITY_SUM'*, *'PRODUCTSTANDARDCOST_AVG'*, *'SALESAMOUNT_SUM'*, *'TAXAMT_SUM'*, *'TOTALPRODUCTCOST_SUM'*,*'UNITPRICEDISCOUNTPCT_AVG'*, *'UNITPRICE_AVG'*, *'UNITPRICE_SUM'*
over *1*, *2*, *3*, and *12* weeks.

In [15]:
salesds = sales_promo.datetrunc(dates={'ORDERDATE': 'week'}).rename(
                                renames={'ORDERDATE_WEEK': 'ORDERWEEK'}).aggregate(
                                group_by=['PRODUCTKEY', 'ORDERWEEK'],
                                aggregations={'DISCOUNTAMOUNT': ['MIN', 'MAX', 'AVG', 'SUM'], 
                                              'DISCOUNTPCT': ['MIN', 'MAX', 'AVG', 'SUM'],
                                              'ORDERQUANTITY': ['SUM'],
                                              'PRODUCTSTANDARDCOST': ['AVG', 'SUM'],
                                              'SALESAMOUNT': ['SUM'], 
                                              'TAXAMT': ['SUM'],
                                              'TOTALPRODUCTCOST': ['AVG', 'SUM'],
                                              'UNITPRICE': ['AVG', 'SUM'],
                                              'UNITPRICEDISCOUNTPCT': ['MIN', 'MAX', 'AVG', 'SUM']}).lag(
                                columns=['DISCOUNTAMOUNT_AVG', 'DISCOUNTPCT_AVG', 'ORDERQUANTITY_SUM', 
                                         'PRODUCTSTANDARDCOST_AVG', 'SALESAMOUNT_SUM', 'TAXAMT_SUM', 
                                         'TOTALPRODUCTCOST_SUM','UNITPRICEDISCOUNTPCT_AVG', 
                                         'UNITPRICE_AVG', 'UNITPRICE_SUM'],
                                amounts=[1, 2, 3, 12],
                                order_by=['PRODUCTKEY', 'ORDERWEEK'],
                                partition=['PRODUCTKEY'])
   
salesds.order(col_list=['PRODUCTKEY', 'ORDERWEEK'], order_method="ASC").preview()

,PRODUCTKEY,ORDERWEEK,DISCOUNTAMOUNT_MIN,DISCOUNTAMOUNT_MAX,DISCOUNTAMOUNT_AVG,DISCOUNTAMOUNT_SUM,DISCOUNTPCT_MIN,DISCOUNTPCT_MAX,DISCOUNTPCT_AVG,DISCOUNTPCT_SUM,...,LAG_UNITPRICEDISCOUNTPCT_AVG_3,LAG_UNITPRICEDISCOUNTPCT_AVG_12,LAG_UNITPRICE_AVG_1,LAG_UNITPRICE_AVG_2,LAG_UNITPRICE_AVG_3,LAG_UNITPRICE_AVG_12,LAG_UNITPRICE_SUM_1,LAG_UNITPRICE_SUM_2,LAG_UNITPRICE_SUM_3,LAG_UNITPRICE_SUM_12
0,214,2012-12-23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,None,None,None,None,None,None,None,None
1,214,2012-12-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,34.9900000000,None,None,None,139.9600,None,None,None
2,214,2013-01-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,34.9900000000,34.9900000000,None,None,349.9000,139.9600,None,None
3,214,2013-01-13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,34.9900000000,34.9900000000,34.9900000000,None,419.8800,349.9000,139.9600,None
4,214,2013-01-20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,34.9900000000,34.9900000000,34.9900000000,None,209.9400,419.8800,349.9000,None
5,214,2013-01-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,34.9900000000,34.9900000000,34.9900000000,None,384.8900,209.9400,419.8800,None
6,214,2013-02-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,34.9900000000,34.9900000000,34.9900000000,None,1084.6900,384.8900,209.9400,None
7,214,2013-02-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,34.9900000000,34.9900000000,34.9900000000,None,1539.5600,1084.6900,384.8900,None
8,214,2013-02-17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,34.9900000000,34.9900000000,34.9900000000,None,1189.6600,1539.5600,1084.6900,None
9,214,2013-02-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,34.9900000000,34.9900000000,34.9900000000,None,1154.6700,1189.6600,1539.5600,None


In addition to lag variables, the moving average of the quantites can be useful. In this case, we'll calculate the moving average over *4* observations of *ORDERQUANTITY_SUM* and *TOTALPRODUCTCOST_SUM* using the transform `moving_avg`.

In [16]:
salesds = sales_promo.datetrunc(dates={'ORDERDATE': 'week'}).rename(
                                renames={'ORDERDATE_WEEK': 'ORDERWEEK'}).aggregate(
                                group_by=['PRODUCTKEY', 'ORDERWEEK'],
                                aggregations={'DISCOUNTAMOUNT': ['MIN', 'MAX', 'AVG', 'SUM'], 
                                              'DISCOUNTPCT': ['MIN', 'MAX', 'AVG', 'SUM'],
                                              'ORDERQUANTITY': ['SUM'],
                                              'PRODUCTSTANDARDCOST': ['AVG', 'SUM'],
                                              'SALESAMOUNT': ['SUM'], 
                                              'TAXAMT': ['SUM'],
                                              'TOTALPRODUCTCOST': ['AVG', 'SUM'],
                                              'UNITPRICE': ['AVG', 'SUM'],
                                              'UNITPRICEDISCOUNTPCT': ['MIN', 'MAX', 'AVG', 'SUM']}).lag(
                                columns=['DISCOUNTAMOUNT_AVG', 'DISCOUNTPCT_AVG', 'ORDERQUANTITY_SUM', 
                                         'PRODUCTSTANDARDCOST_AVG', 'SALESAMOUNT_SUM', 'TAXAMT_SUM', 
                                         'TOTALPRODUCTCOST_SUM','UNITPRICEDISCOUNTPCT_AVG', 
                                         'UNITPRICE_AVG', 'UNITPRICE_SUM'],
                                amounts=[1,   , 3,    ],
                                order_by=['PRODUCTKEY', 'ORDERWEEK'],
                                partition=['PRODUCTKEY']).             (
                                input_columns=['ORDERQUANTITY_SUM', 'SALESAMOUNT_SUM'],
                                window_sizes=[4],
                                order_by=['PRODUCTKEY', 'ORDERWEEK'],
                                partition=['PRODUCTKEY'])
    
salesds.order(col_list=['PRODUCTKEY', 'ORDERWEEK'], order_method="ASC").preview()

,PRODUCTKEY,ORDERWEEK,DISCOUNTAMOUNT_MIN,DISCOUNTAMOUNT_MAX,DISCOUNTAMOUNT_AVG,DISCOUNTAMOUNT_SUM,DISCOUNTPCT_MIN,DISCOUNTPCT_MAX,DISCOUNTPCT_AVG,DISCOUNTPCT_SUM,...,LAG_UNITPRICE_AVG_1,LAG_UNITPRICE_AVG_2,LAG_UNITPRICE_AVG_3,LAG_UNITPRICE_AVG_12,LAG_UNITPRICE_SUM_1,LAG_UNITPRICE_SUM_2,LAG_UNITPRICE_SUM_3,LAG_UNITPRICE_SUM_12,MEAN_ORDERQUANTITY_SUM_4,MEAN_SALESAMOUNT_SUM_4
0,214,2012-12-23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,None,None,None,None,None,None,None,None,4.000,139.9600000
1,214,2012-12-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,None,None,None,139.9600,None,None,None,7.000,244.9300000
2,214,2013-01-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,34.9900000000,None,None,349.9000,139.9600,None,None,8.666,303.2466666
3,214,2013-01-13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,34.9900000000,34.9900000000,None,419.8800,349.9000,139.9600,None,8.000,279.9200000
4,214,2013-01-20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,34.9900000000,34.9900000000,None,209.9400,419.8800,349.9000,None,9.750,341.1525000
5,214,2013-01-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,34.9900000000,34.9900000000,None,384.8900,209.9400,419.8800,None,15.000,524.8500000
6,214,2013-02-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,34.9900000000,34.9900000000,None,1084.6900,384.8900,209.9400,None,23.000,804.7700000
7,214,2013-02-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,34.9900000000,34.9900000000,None,1539.5600,1084.6900,384.8900,None,30.000,1049.7000000
8,214,2013-02-17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,34.9900000000,34.9900000000,None,1189.6600,1539.5600,1084.6900,None,35.500,1242.1450000
9,214,2013-02-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,34.9900000000,34.9900000000,34.9900000000,None,1154.6700,1189.6600,1539.5600,None,37.000,1294.6300000


#### Save result

At this point, the data has been aggregated to weekly data and multiple transformations have been applied. This could be a good starting point for additional analysis and useful for visualization. For this reason, we will publish it to Rasgo to make it available for others to use. This can be done with the `rasgo.publish.dataset` function

In [21]:
weeklysales = rasgo.publish.dataset(dataset=          ,
                                    name="WKSP: AdventureWorks: weekly sales",
                                    description="Internet Sales data converted to weekly sales")
weeklysales

APIError: Internal API Error when making POST request

Status Code: 503

Internal Error Details: <!DOCTYPE html>
	<html>
	  <head>
		<meta name="viewport" content="width=device-width, initial-scale=1">
		<meta charset="utf-8">
		<title>Application Error</title>
		<style media="screen">
		  html,body,iframe {
			margin: 0;
			padding: 0;
		  }
		  html,body {
			height: 100%;
			overflow: hidden;
		  }
		  iframe {
			width: 100%;
			height: 100%;
			border: 0;
		  }
		</style>
	  </head>
	  <body>
		<iframe src="//www.herokucdn.com/error-pages/application-error.html"></iframe>
	  </body>
	</html>

We can examine this dataset on Rasgo by clicking the link below

In [ ]:
print(f"https://app.rasgoml.com/datasets/{weeklysales.id}")

Using this dataset, we can continue data preparation.

### Create Modeling Data

We can now begin final preparation for modeling with this dataset. To do this, we need to do three things. First, the target (next weeks sales needs to be created). Second, the categorical variables should be one-hot encoded. Finally, missing values should be imputed for the numeric columns.

#### Target Creation

Use the `lag` transform with a negative lag value to get next weeks sales as the target. While doing this, rename the value to make it clear that it is the target.

In [ ]:
modelingds = weeklysales.lag(columns=['SALESAMOUNT_SUM'],
                             amounts=    ,
                             order_by=['PRODUCTKEY', 'ORDERWEEK'],
                                        =['PRODUCTKEY']).rename(
                             renames={'LAG_SALESAMOUNT_SUM__1': 'TARGET_SALESAMOUNT'})
modelingds.preview()

#### Categorical encoding

The only column that needs to be encoded *PRODUCTKEY*. We can use the `one_hot_encode`, `target_encode` or the `label_encode` transforms. 

For features with a large number of categorties and when using tree-based modeling algorithms, the `label_encode` transform is a useful technique to encode categorical variables. The `target_encode` transform replaces the category by the mean target value of that category. Target encoding is a very powerful techinque to encode these high-cardinality categorical variables efficiently and help improve model performance. In this case, we will target encode the product_key

In [ ]:
modelingds = modelingds.                (column='PRODUCTKEY',
                                      target=                         )

modelingds.order(col_list=['PRODUCTKEY', 'ORDERWEEK'], order_method="ASC").preview()

#### Imputation

As a final step before modeling, all numeric columns should have missing values replaced by a number. This can be done by the `impute` transformation. If a linear or logistic regression, SVM or neural network algorithm was going to be applied, we may want to impute the mean or median. This could be done by passing 'mean' or 'median' in through the imputations dictionary.

As the modeling algoritm applied here is tree-based, we can simply impute and extreme value. All of the features created are non-negative or close to zero, so we will impute a very large negative number, *-999,999*.

In [ ]:
imputation_dict = {'DISCOUNTAMOUNT_AVG': -999999,
                   'DISCOUNTAMOUNT_MAX': -999999,
                   'DISCOUNTAMOUNT_MIN': -999999,
                   'DISCOUNTAMOUNT_SUM': -999999,
                   'DISCOUNTPCT_AVG': -999999,
                   'DISCOUNTPCT_MAX': -999999,
                   'DISCOUNTPCT_MIN': -999999,
                   'DISCOUNTPCT_SUM': -999999,
                   'LAG_DISCOUNTAMOUNT_AVG_1': -999999,
                   'LAG_DISCOUNTAMOUNT_AVG_12': -999999,
                   'LAG_DISCOUNTAMOUNT_AVG_2': -999999,
                   'LAG_DISCOUNTAMOUNT_AVG_3': -999999,
                   'LAG_DISCOUNTPCT_AVG_1': -999999,
                   'LAG_DISCOUNTPCT_AVG_12': -999999,
                   'LAG_DISCOUNTPCT_AVG_2': -999999,
                   'LAG_DISCOUNTPCT_AVG_3': -999999,
                   'LAG_ORDERQUANTITY_SUM_1': -999999,
                   'LAG_ORDERQUANTITY_SUM_12': -999999,
                   'LAG_ORDERQUANTITY_SUM_2': -999999,
                   'LAG_ORDERQUANTITY_SUM_3': -999999,
                   'LAG_PRODUCTSTANDARDCOST_AVG_1': -999999,
                   'LAG_PRODUCTSTANDARDCOST_AVG_12': -999999,
                   'LAG_PRODUCTSTANDARDCOST_AVG_2': -999999,
                   'LAG_PRODUCTSTANDARDCOST_AVG_3': -999999,
                   'LAG_SALESAMOUNT_SUM_1': -999999,
                   'LAG_SALESAMOUNT_SUM_12': -999999,
                   'LAG_SALESAMOUNT_SUM_2': -999999,
                   'LAG_SALESAMOUNT_SUM_3': -999999,
                   'LAG_TAXAMT_SUM_1': -999999,
                   'LAG_TAXAMT_SUM_12': -999999,
                   'LAG_TAXAMT_SUM_2': -999999,
                   'LAG_TAXAMT_SUM_3': -999999,
                   'LAG_TOTALPRODUCTCOST_SUM_1': -999999,
                   'LAG_TOTALPRODUCTCOST_SUM_12': -999999,
                   'LAG_TOTALPRODUCTCOST_SUM_2': -999999,
                   'LAG_TOTALPRODUCTCOST_SUM_3': -999999,
                   'LAG_UNITPRICEDISCOUNTPCT_AVG_1': -999999,
                   'LAG_UNITPRICEDISCOUNTPCT_AVG_12': -999999,
                   'LAG_UNITPRICEDISCOUNTPCT_AVG_2': -999999,
                   'LAG_UNITPRICEDISCOUNTPCT_AVG_3': -999999,
                   'LAG_UNITPRICE_AVG_1': -999999,
                   'LAG_UNITPRICE_AVG_12': -999999,
                   'LAG_UNITPRICE_AVG_2': -999999,
                   'LAG_UNITPRICE_AVG_3': -999999,
                   'LAG_UNITPRICE_SUM_1': -999999,
                   'LAG_UNITPRICE_SUM_12': -999999,
                   'LAG_UNITPRICE_SUM_2': -999999,
                   'LAG_UNITPRICE_SUM_3': -999999,
                   'MEAN_ORDERQUANTITY_SUM_4': -999999,
                   'MEAN_SALESAMOUNT_SUM_4': -999999,
                   'ORDERQUANTITY_SUM': -999999,
                   'PRODUCTSTANDARDCOST_AVG': -999999,
                   'PRODUCTSTANDARDCOST_SUM': -999999,
                   'SALESAMOUNT_SUM': -999999,
                   'TAXAMT_SUM': -999999,
                   'TOTALPRODUCTCOST_AVG': -999999,
                   'TOTALPRODUCTCOST_SUM': -999999,
                   'UNITPRICEDISCOUNTPCT_AVG': -999999,
                   'UNITPRICEDISCOUNTPCT_MAX': -999999,
                   'UNITPRICEDISCOUNTPCT_MIN': -999999,
                   'UNITPRICEDISCOUNTPCT_SUM': -999999,
                   'UNITPRICE_AVG': -999999,
                   'UNITPRICE_SUM': -999999}

In [ ]:
modelingds = modelingds.        (imputations=imputation_dict)

modelingds.order(col_list=[                , 'ORDERWEEK'], order_method="ASC").preview()

#### Train-test split

As this is a time-series problem, a random train-test split won't work as there will be leakage from observations near the end of the time frame in the training set to observations earlier than this in the test set. The way to avoid this problem is to perform the split based on the date. The transformation `train_test_split` can do this by passing the date columns through the parameter **order_by**.

In [ ]:
modelingds = modelingds.train_test_split(order_by=['ORDERWEEK'],
                                                               )
    
modelingds.order(col_list=['PRODUCTKEY', 'ORDERWEEK'], order_method="ASC").preview()

#### Save Modeling Dataset

We can now save this modeling dataset so we can return to it in the future.

In [ ]:
modeling = rasgo.         .dataset(dataset=modelingds,
                                 name="WKSP: AdventureWorks: Sales Forecast Modeling",
                                 description="Modeling dataset for Internet Sales Forecasting")
modeling

We can examine this dataset on Rasgo by clicking the link below

In [ ]:
print(f"https://app.rasgoml.com/datasets/{modeling.id}")

Capture this dataset ID for use in prediction.

In [ ]:
ds_id =           .id

### Modeling

We are now ready to build the model. First, get the modeling data from Rasgo using `to_df`.

In [ ]:
df = modeling.      ().reset_index(drop=True)
df = df[~df.ORDERWEEK.isna()]

Check for numeric datatypes and convert the numeric ones to floats.

In [ ]:
for c in df.select_dtypes(exclude=[np.number]).columns:
    if c not in [              , 'TT_SPLIT']:
        df[c] = pd.to_numeric(df[c])

Eliminate the last week of data as there is no target.

In [ ]:
df = df[~df.                       .isna()]

#### Train the model

First, split the data using the TT_SPLIT column.

In [ ]:
df_train = df[df[              ] == 'TRAIN'].drop(columns=['TT_SPLIT', 'ORDERWEEK'])
df_test = df[df['TT_SPLIT'] ==           ].drop(columns=['TT_SPLIT', 'ORDERWEEK'])

In [ ]:
y_train = df_train['TARGET_SALESAMOUNT']
X_train = df_train.drop(columns=['TARGET_SALESAMOUNT'])
y_test = df_test['TARGET_SALESAMOUNT']
X_test = df_test.drop(columns=['TARGET_SALESAMOUNT'])

#### Fit the model

For illustration purposes, we are just fitting the model with a single set of parameters. In general, you should optimize the hyperparameters before building the final model. That process is beyond the scope of this document.

In [ ]:
model = xgb.XGBRegressor(n_estimators=     ,
                         max_depth=5,
                         eta=0.01,
                         random_state=1066,
                         subsample=0.7,
                         colsample_bytree=0.7)

model.fit(           , y_train)

#### Check the performance

In [ ]:
model.predict(         )

In [ ]:
rmse = np.sqrt(                      (y_test, model.predict(X_test)))
rmse

### Predict on new data

Since our feature engineering was saved in Rasgo, as new data enters the system, it will automatically be prepared for modeling. We can just pull the data in question and make a prediction on it.

In this case, if we are making these predictions each week, we can just pull the most recent week. In this particular data, that is '*2014-01-19*'.

#### Pull the data

Use to_df to grab the data from this date. We have several columns not needed in the model, so we will drop those as well.

In [3]:
predictdf = rasgo.get.dataset(ds_id).wto_df(filters={"ORDERWEEK":"2014-01-19"})
for c in predictdf.select_dtypes(exclude=[np.number]).columns:
    if c not in [             , 'TT_SPLIT']:
        predictdf[c] = pd.to_numeric(predictdf[c])
predictdf.head()

SyntaxError: invalid syntax (628212070.py, line 3)

Now use the model to get the sales forecast. We will create a dataframe to hold the predictions then drop the columns not needed by the model before making the prediction.

In [ ]:
salesforecastdf = predictdf[['PRODUCTKEY', 'ORDERWEEK']].copy()
salesforecastdf[            ] = model.predict(predictdf.drop(columns=['TT_SPLIT', 'ORDERWEEK', 'TARGET_SALESAMOUNT']))
salesforecastdf